# Validating RAS Mapper Layers and Terrain Files

In [ ]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
# Set USE_LOCAL_SOURCE based on your setup:
#   True  = Use local source code (for developers editing ras-commander)  
#   False = Use pip-installed package (for users)
# =============================================================================

USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

# -----------------------------------------------------------------------------
if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)  # Parent of examples/ = repo root
    if local_path not in sys.path:
        sys.path.insert(0, local_path)  # Insert at position 0 = highest priority
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander (will use local or pip based on toggle above)
from ras_commander import *
from ras_commander.validation_base import ValidationSeverity

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

---

## Overview

This notebook demonstrates the RAS Mapper layer validation framework in ras-commander. RAS Mapper uses various geospatial layers (terrain, land cover, boundaries) to visualize and configure HEC-RAS models. Validating these layers ensures:

- Files exist and are readable
- Formats are supported (GeoJSON, Shapefile, GeoTIFF, HDF)
- Coordinate reference systems (CRS) are defined
- Raster metadata is valid
- Spatial extents cover the model domain

The validation framework provides:
- Format validation
- CRS validation
- Raster metadata checks
- Spatial extent verification
- Specialized terrain and land cover validation
- Comprehensive validation reports

We'll use the **Muncie** example project which includes RAS Mapper layers.

---

## Extract Example Project

In [ ]:
# Extract Muncie project (contains RAS Mapper configuration)
project_path = RasExamples.extract_project("Muncie")
print(f"\nProject extracted to: {project_path}")

In [ ]:
# Find geospatial files in the project
from pathlib import Path

# Common geospatial file extensions
geo_extensions = ['*.tif', '*.tiff', '*.shp', '*.geojson', '*.json', '*.hdf', '*.h5']

geo_files = []
for pattern in geo_extensions:
    geo_files.extend(project_path.glob(pattern))
    # Also check subdirectories
    geo_files.extend(project_path.glob(f"*/{pattern}"))
    geo_files.extend(project_path.glob(f"*/*/{pattern}"))

print(f"Found {len(geo_files)} geospatial file(s):")
for geo_file in geo_files[:10]:  # Show first 10
    size_kb = geo_file.stat().st_size / 1024
    print(f"  - {geo_file.relative_to(project_path)} ({size_kb:.2f} KB)")
if len(geo_files) > 10:
    print(f"  ... and {len(geo_files) - 10} more")

In [ ]:
# Look for terrain files specifically
terrain_files = [f for f in geo_files if 'terrain' in f.name.lower() or f.suffix.lower() in ['.tif', '.tiff']]

if terrain_files:
    test_terrain_file = terrain_files[0]
    print(f"Using terrain file: {test_terrain_file.name}")
else:
    print("⚠️ No terrain files found - will use synthetic examples")
    test_terrain_file = project_path / "terrain.tif"  # For examples

---

## 1. Format Validation: `check_layer_format()`

Validates that layer files exist, are readable, and have supported formats.

In [ ]:
# Example 1: Valid terrain file (GeoTIFF)
if terrain_files:
    result = RasMap.check_layer_format(test_terrain_file)
    print(f"File: {test_terrain_file.name}")
    print(f"Result: {result}")
    print(f"\nDetails:")
    for key, value in result.details.items():
        print(f"  {key}: {value}")

In [ ]:
# Example 2: Check various file types
file_types_to_check = {
    'GeoTIFF': [f for f in geo_files if f.suffix.lower() in ['.tif', '.tiff']],
    'Shapefile': [f for f in geo_files if f.suffix.lower() == '.shp'],
    'GeoJSON': [f for f in geo_files if f.suffix.lower() in ['.geojson', '.json']],
    'HDF': [f for f in geo_files if f.suffix.lower() in ['.hdf', '.h5']]
}

print("Format validation by file type:\n")
for file_type, files in file_types_to_check.items():
    if files:
        test_file = files[0]
        result = RasMap.check_layer_format(test_file)
        status = "✓" if result.passed else "✗"
        print(f"{status} {file_type}: {test_file.name}")
        print(f"  {result.message}")
        if result.severity == ValidationSeverity.WARNING:
            print(f"  ⚠️ {result.severity.value.upper()}")
        print()

In [ ]:
# Example 3: Invalid file (doesn't exist)
nonexistent_file = project_path / "missing_terrain.tif"

result = RasMap.check_layer_format(nonexistent_file)
print(f"File: {nonexistent_file.name}")
print(f"Result: {result}")
print(f"Passed: {result.passed}")
print(f"Severity: {result.severity.value}")

---

## 2. CRS Validation: `check_layer_crs()`

Validates coordinate reference system and checks compatibility with project CRS.

In [ ]:
# Example 1: Check terrain CRS
if terrain_files:
    result = RasMap.check_layer_crs(test_terrain_file)
    print(f"File: {test_terrain_file.name}")
    print(f"Result: {result}")
    print(f"\nDetails:")
    for key, value in result.details.items():
        print(f"  {key}: {value}")

In [ ]:
# Example 2: Check CRS compatibility with expected projection
if terrain_files:
    # First, get the CRS from the terrain file
    result1 = RasMap.check_layer_crs(test_terrain_file)
    
    if result1.passed and 'crs' in result1.details:
        terrain_crs = result1.details['crs']
        print(f"Terrain CRS: {terrain_crs}")
        
        # Check if another file matches
        if len(geo_files) > 1:
            other_file = [f for f in geo_files if f != test_terrain_file][0]
            result2 = RasMap.check_layer_crs(other_file, expected_crs=terrain_crs)
            
            print(f"\nComparing with: {other_file.name}")
            print(f"Result: {result2}")
            if result2.passed:
                print("  ✓ CRS matches terrain file")
            else:
                print("  ✗ CRS mismatch - may need reprojection")

---

## 3. Raster Metadata: `check_raster_metadata()`

Validates raster-specific properties (dimensions, resolution, no-data values).

In [ ]:
# Example 1: Check terrain raster metadata
if terrain_files:
    result = RasMap.check_raster_metadata(test_terrain_file)
    print(f"File: {test_terrain_file.name}")
    print(f"Result: {result}")
    print(f"\nDetails:")
    for key, value in result.details.items():
        print(f"  {key}: {value}")

In [ ]:
# Example 2: Check multiple raster files
raster_files = [f for f in geo_files if f.suffix.lower() in ['.tif', '.tiff']]

if raster_files:
    print(f"Raster metadata for {len(raster_files)} file(s):\n")
    
    for raster_file in raster_files[:3]:  # Check first 3
        result = RasMap.check_raster_metadata(raster_file)
        print(f"📁 {raster_file.name}")
        
        if result.passed and result.details:
            print(f"  Dimensions: {result.details.get('width')} × {result.details.get('height')} pixels")
            print(f"  Resolution: {result.details.get('pixel_size_x')} × {result.details.get('pixel_size_y')}")
            print(f"  Bands: {result.details.get('band_count')}")
            if 'nodata_value' in result.details:
                print(f"  No-data value: {result.details['nodata_value']}")
        else:
            print(f"  {result.message}")
        print()

---

## 4. Spatial Extent: `check_spatial_extent()`

Validates spatial coverage and checks overlap with expected domain.

In [ ]:
# Example 1: Check terrain spatial extent
if terrain_files:
    result = RasMap.check_spatial_extent(test_terrain_file)
    print(f"File: {test_terrain_file.name}")
    print(f"Result: {result}")
    print(f"\nDetails:")
    for key, value in result.details.items():
        if key == 'bounds':
            print(f"  {key}:")
            for bound_key, bound_val in value.items():
                print(f"    {bound_key}: {bound_val}")
        else:
            print(f"  {key}: {value}")

In [ ]:
# Example 2: Check coverage overlap between layers
if len(geo_files) >= 2:
    file1 = geo_files[0]
    file2 = geo_files[1]
    
    # Get extent of first file
    result1 = RasMap.check_spatial_extent(file1)
    if result1.passed and 'bounds' in result1.details:
        expected_bounds = result1.details['bounds']
        
        # Check if second file overlaps
        result2 = RasMap.check_spatial_extent(file2, expected_bounds=expected_bounds)
        
        print(f"Checking overlap:\n")
        print(f"File 1: {file1.name}")
        print(f"  Bounds: {expected_bounds}")
        print(f"\nFile 2: {file2.name}")
        print(f"  {result2}")
        
        if result2.passed:
            print(f"\n  ✓ Files have overlapping coverage")
        else:
            print(f"\n  ✗ Coverage mismatch detected")

---

## 5. Terrain Layer Validation: `check_terrain_layer()`

Specialized validation for terrain/elevation layers.

In [ ]:
# Example 1: Comprehensive terrain validation
if terrain_files:
    result = RasMap.check_terrain_layer(test_terrain_file)
    print(f"Terrain file: {test_terrain_file.name}")
    print(f"Result: {result}")
    print(f"\nDetails:")
    for key, value in result.details.items():
        if isinstance(value, dict):
            print(f"  {key}:")
            for sub_key, sub_val in value.items():
                print(f"    {sub_key}: {sub_val}")
        else:
            print(f"  {key}: {value}")

In [ ]:
# Example 2: Validate elevation range is reasonable
if terrain_files:
    # Get terrain statistics
    result = RasMap.check_terrain_layer(test_terrain_file)
    
    if result.passed and 'elevation_stats' in result.details:
        stats = result.details['elevation_stats']
        min_elev = stats.get('min')
        max_elev = stats.get('max')
        mean_elev = stats.get('mean')
        
        print(f"Terrain elevation statistics:\n")
        print(f"  Minimum: {min_elev:.2f} ft" if min_elev is not None else "  Minimum: N/A")
        print(f"  Maximum: {max_elev:.2f} ft" if max_elev is not None else "  Maximum: N/A")
        print(f"  Mean: {mean_elev:.2f} ft" if mean_elev is not None else "  Mean: N/A")
        
        # Check for reasonable range (example: US elevations typically -300 to 14,500 ft)
        if min_elev is not None and max_elev is not None:
            if min_elev < -500 or max_elev > 15000:
                print("\n  ⚠️ WARNING: Elevation range may be unrealistic for US terrain")
                print("     Check units (should be feet or meters, not mm or other)")
            else:
                print("\n  ✓ Elevation range appears reasonable")

---

## 6. Land Cover Validation: `check_land_cover_layer()`

Specialized validation for land cover/Manning's n layers.

In [ ]:
# Example 1: Check land cover layer
land_cover_files = [f for f in geo_files if 'land' in f.name.lower() or 'cover' in f.name.lower() or 'manning' in f.name.lower()]

if land_cover_files:
    land_cover_file = land_cover_files[0]
    result = RasMap.check_land_cover_layer(land_cover_file)
    
    print(f"Land cover file: {land_cover_file.name}")
    print(f"Result: {result}")
    print(f"\nDetails:")
    for key, value in result.details.items():
        print(f"  {key}: {value}")
else:
    print("No land cover files found in project")
    print("\nLand cover validation would check:")
    print("  - File format (typically raster)")
    print("  - CRS matches terrain")
    print("  - Categorical data (land use classes)")
    print("  - Spatial coverage matches model domain")

---

## 7. Comprehensive Layer Validation: `check_layer()`

Performs all applicable validation checks and returns a comprehensive report.

In [ ]:
# Example 1: Comprehensive validation of terrain layer
if terrain_files:
    report = RasMap.check_layer(
        layer_file=test_terrain_file,
        layer_type='terrain'
    )
    
    print(f"Comprehensive validation for: {test_terrain_file.name}")
    print(f"\nReport: {report}")
    print(f"Is valid: {report.is_valid}")
    print(f"Has warnings: {report.has_warnings}")
    
    # Print formatted report
    report.print_report(show_passed=True)

In [ ]:
# Example 2: Filter validation results by severity
if terrain_files:
    report = RasMap.check_layer(test_terrain_file, layer_type='terrain')
    
    # Get only errors
    errors = report.get_results_by_severity(ValidationSeverity.ERROR)
    print(f"Errors found: {len(errors)}")
    for error in errors:
        print(f"  ✗ {error}")
    
    # Get only warnings
    warnings = report.get_results_by_severity(ValidationSeverity.WARNING)
    print(f"\nWarnings found: {len(warnings)}")
    for warning in warnings:
        print(f"  ⚠️ {warning}")
    
    # Get info messages
    info = report.get_results_by_severity(ValidationSeverity.INFO)
    print(f"\nInfo messages: {len(info)}")
    for msg in info:
        print(f"  ℹ️ {msg}")

---

## 8. Boolean Convenience: `is_valid_layer()`

Quick boolean check for layer validity.

In [ ]:
# Example: Quick validation of multiple layers
if geo_files:
    print("Quick validation of geospatial layers:\n")
    
    for geo_file in geo_files[:5]:  # Check first 5 files
        is_valid = RasMap.is_valid_layer(geo_file)
        status = "✓ VALID" if is_valid else "✗ INVALID"
        print(f"{status}: {geo_file.name}")

---

## 9. Practical Use Case: Pre-Flight Check for Model Setup

Validate all required RAS Mapper layers before model setup.

In [ ]:
# Simulate pre-flight check for model setup
print("=" * 80)
print("PRE-FLIGHT CHECK: RAS Mapper Layer Validation")
print("=" * 80)

# Required layers for typical 2D model
required_layers = {
    'Terrain': [f for f in terrain_files] if terrain_files else [],
    'Geometry': [f for f in geo_files if f.suffix.lower() in ['.shp', '.geojson']],
}

all_valid = True

for layer_type, files in required_layers.items():
    print(f"\n{'='*80}")
    print(f"Checking {layer_type} Layer(s)")
    print(f"{'='*80}")
    
    if not files:
        print(f"  ✗ No {layer_type.lower()} files found")
        all_valid = False
        continue
    
    for i, layer_file in enumerate(files[:3], 1):  # Check first 3 of each type
        print(f"\n[{i}] {layer_file.name}")
        
        # Quick check
        is_valid = RasMap.is_valid_layer(layer_file)
        
        if is_valid:
            print(f"  ✓ PASS")
            
            # Get additional details
            format_result = RasMap.check_layer_format(layer_file)
            if format_result.details:
                fmt = format_result.details.get('format', 'unknown')
                print(f"    Format: {fmt}")
        else:
            print(f"  ✗ FAIL - running detailed diagnostics...")
            
            # Get detailed report
            layer_type_key = 'terrain' if layer_type == 'Terrain' else None
            report = RasMap.check_layer(layer_file, layer_type=layer_type_key)
            report.print_report(show_passed=False)
            
            all_valid = False

print("\n" + "=" * 80)
if all_valid:
    print("✓ PRE-FLIGHT CHECK PASSED - All required layers valid")
    print("  Ready to configure RAS Mapper")
else:
    print("✗ PRE-FLIGHT CHECK FAILED - Fix layer issues before proceeding")
print("=" * 80)

---

## 10. Visualization: Layer Extent Comparison (Optional)

Visualize spatial extents of multiple layers to verify coverage.

In [ ]:
# Optional: Visualize layer extents using matplotlib
try:
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    
    if geo_files:
        fig, ax = plt.subplots(figsize=(10, 8))
        
        colors = ['red', 'blue', 'green', 'orange', 'purple']
        
        for i, geo_file in enumerate(geo_files[:5]):  # Plot first 5
            result = RasMap.check_spatial_extent(geo_file)
            
            if result.passed and 'bounds' in result.details:
                bounds = result.details['bounds']
                
                # Extract bounds
                minx = bounds.get('minx', 0)
                miny = bounds.get('miny', 0)
                maxx = bounds.get('maxx', 0)
                maxy = bounds.get('maxy', 0)
                
                width = maxx - minx
                height = maxy - miny
                
                # Draw rectangle
                rect = patches.Rectangle(
                    (minx, miny), width, height,
                    linewidth=2, edgecolor=colors[i % len(colors)],
                    facecolor='none', label=geo_file.name
                )
                ax.add_patch(rect)
        
        ax.set_xlabel('Easting')
        ax.set_ylabel('Northing')
        ax.set_title('Spatial Extents of Geospatial Layers')
        ax.legend(loc='upper right')
        ax.set_aspect('equal')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        print("\n✓ Extent visualization complete")
        print("  Overlapping rectangles indicate good coverage alignment")
    
except ImportError:
    print("matplotlib not available - skipping visualization")

---

## Cleanup

In [ ]:
# Clean up extracted project
import shutil

if project_path.parent.name == "example_projects":
    shutil.rmtree(project_path.parent, ignore_errors=True)
    print("✓ Cleaned up example projects")

---

## Summary

This notebook demonstrated the RAS Mapper layer validation framework:

**Individual Validation Methods**:
- `check_layer_format()` - Validates file format and accessibility
- `check_layer_crs()` - Validates coordinate reference system
- `check_raster_metadata()` - Validates raster properties
- `check_spatial_extent()` - Validates spatial coverage

**Specialized Validations**:
- `check_terrain_layer()` - Terrain-specific validation
- `check_land_cover_layer()` - Land cover-specific validation

**Comprehensive Validation**:
- `check_layer()` - Runs all applicable checks, returns ValidationReport

**Boolean Convenience**:
- `is_valid_layer()` - Quick validity check

**Validation Reports**:
- Filter by severity (INFO, WARNING, ERROR, CRITICAL)
- Get failed checks
- Print formatted reports

Use these tools to ensure RAS Mapper layers are valid before model setup and execution!